# Libraries

본 실습에서는 다음 두개의 Python 라이브러리들을 사용합니다.

1. tqdm: 프로그레스바를 표시해 주는 라이브러리 ([참조링크](https://github.com/tqdm/tqdm))

1. pytube: YouTube에서 video 컨텐츠를 다운로드 할 수 있도록 해주는 라이브러리 ([참조링크](https://pytube.io/en/latest/index.html))

## tqdm Overview

In [1]:
from tqdm import tqdm
from tqdm import trange
from time import sleep

In [2]:
# 다음 코드는 0.05초 마다 한번 씩 프로그레스 바를 업데이트 합니다.
for _ in tqdm(range(100)):
    sleep(0.05)

100%|██████████| 100/100 [00:05<00:00, 19.57it/s]


In [3]:
# tqdm을 객체로 선언하고 update()를 호출해서 프로그레스를 업데이트 할 수도 있습니다.
pbar = tqdm(total=100)

for _ in range(10):
    sleep(0.1)
    pbar.update(10)
pbar.close()


100%|██████████| 100/100 [00:01<00:00, 97.86it/s]


In [4]:
# 혹은, with keyword를 사용해서 close() 호출을 없앨 수도 있습니다.
with tqdm(total=100) as qbar:
    for _ in range(10):
        sleep(0.1)
        qbar.update(10)

100%|██████████| 100/100 [00:01<00:00, 97.96it/s]


## pytube overview
시간이 오래 걸리는 일을 모사하기 위해 pytube로 YouTube video를 다운로드 하는 경우를 가정했습니다.


In [23]:
from pytube import YouTube
from IPython.display import Image

In [24]:
# YouTube clip links

# 과학쿠키 - 초전도체
super_conductor = "https://www.youtube.com/watch?v=d_F7MVuJJu8"

# 8마일 랩배틀 에미넴모음, 연령제한
eight_miles = "https://www.youtube.com/watch?v=tWFejQSKIYg"

# IVE 아이브 'I AM' MV 
ive_mv = "https://www.youtube.com/watch?v=6ZUIwj3FgUY&list=PLObJu56es5eN3sNytFyk7CA3HKgXoRvf6"

In [25]:

url = ive_mv
youtube_clip = YouTube(url) # 유튜브 객체 생성

In [26]:
# Video clip에 대한 더 많은 정보는 다음의 링크를 참조 하세요.
# https://pytube.io/en/latest/api.html?highlight=author#youtube-object
print(f"제목:\t{youtube_clip.title}")
print(f"설명:\t" "<없음>" if youtube_clip.description is None else "f{youtube_clip.description}")
print(f"길이:\t약 {youtube_clip.length//60}분")     # /는 밑에 소수점이 나오고 //는 밑에 소수점을 버린 정수형으로 표현
print(f"연령제한:\t" "없음" if youtube_clip.age_restricted is False else "<제한됨>")

Image(url=youtube_clip.thumbnail_url, width=500)
print(f"{youtube_clip.thumbnail_url}")  # 유튜브 객체 썸네일

# 유튜브 객체 youtube_clip의 정보들을 확인 가능 (위에는 간단한것들이 더 많은 정보들을 볼수있다.)

제목:	I AM
설명:	<없음>
길이:	약 3분
연령제한:	없음
https://i.ytimg.com/vi/6ZUIwj3FgUY/hq720.jpg?sqp=-oaymwEXCNUGEOADIAQqCwjVARCqCBh4INgESFo&rs=AOn4CLAbiga7YTfXymBrimS9Q0PtbQP7yA


In [27]:
# Streams
for i, item in enumerate(youtube_clip.streams): # streams의 목록들을 인덱스와 item로 반환
    print(f"[1]: {item}")   # [1] {i}


AgeRestrictedError: 6ZUIwj3FgUY is age restricted, and can't be accessed without logging in.

In [22]:
# 특정한 비디오 스트림을 지정해서 다운로드를 수행합니다. 필터링하는 다양한 방법들은 아래의 링크를 참조하세요.
# https://pytube.io/en/latest/api.html?highlight=highest%20resolution#streamquery-object

download_dir = "videos"

# E.g. download the first in progressive mp4 file.
#youtube_clip.streams.filter(progressive=True, file_extension='mp4').first().download(download_dir)

# E.g. download a highest resolution video.
youtube_clip.streams.get_highest_resolution().download(download_dir)

Downloaded!


'/home/intel/Desktop/intel/kcci.intel.ai.project/Class02/python-parallelism/videos/출시 1년도 되지 않은 카트라이더 드리프트는 왜 망해갈까 feat리버스.mp4'

### Callbacks

다운로드의 진행과 완료때 호출 될 callback을 지정할 수 있습니다. 다음과 같이 생성자에서 on_progress_callback 혹은 on_complete_callback에 callback 함수를 연결해 주면 됩니다.

```
class YouTube:
    """Core developer interface for pytube."""

    def __init__(
        self,
        url: str,
        on_progress_callback: Optional[Callable[[Any, bytes, int], None]] = None,
        on_complete_callback: Optional[Callable[[Any, Optional[str]], None]] = None,
        proxies: Dict[str, str] = None,
        use_oauth: bool = False,
        allow_oauth_cache: bool = True
    ):
```

In [20]:
# 출시 1년도 되지 않은 카트라이더 드리프트는 왜 망해갈까
video_url = "https://www.youtube.com/watch?v=r8y7RjlhkNE"

# Progress callback
def on_progress(stream, chunk, bytes_remaining):
    print(f"Progress: about {bytes_remaining // (1024*1024)} MB remaining")

# Completion callback
def on_complete(stream, file_handle):
    print("Downloaded!")

youtube_clip = YouTube(video_url,
                       on_progress_callback=on_progress,
                       on_complete_callback=on_complete)

youtube_clip.streams.get_highest_resolution().download("videos")

Progress: about 71 MB remaining
Progress: about 62 MB remaining
Progress: about 53 MB remaining
Progress: about 44 MB remaining
Progress: about 35 MB remaining
Progress: about 26 MB remaining
Progress: about 17 MB remaining
Progress: about 8 MB remaining
Progress: about 0 MB remaining
Downloaded!


'/home/intel/Desktop/intel/kcci.intel.ai.project/Class02/python-parallelism/videos/출시 1년도 되지 않은 카트라이더 드리프트는 왜 망해갈까 feat리버스.mp4'